env: scGPT

# import

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

import pickle
import sys
import requests

from types import MethodType
import importlib
from scperturb import *

import anndata as ad

In [2]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

import celloracle as co
co.__version__

'0.18.0'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from tqdm import tqdm

In [4]:
sys.path.append("/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/")

In [5]:
import v1
from v1.utils import *
from v1.dataloader import *

In [6]:
importlib.reload(v1)
importlib.reload(v1.utils)
importlib.reload(v1.dataloader)

<module 'v1.dataloader' from '/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/dataloader.py'>

# L1000运行CellOracle-非并行版

In [7]:
# - get cell line name
common_cell_line = \
{   'A549': 'A549',
    'HEPG2': 'HepG2',
    'HT29': 'HT29',
    'MCF7': 'MCF7',
    # 'SKBR3': 'SK-BR-3',
    'SW480': 'SW480',
    'PC3': 'PC3',
    'A375': 'A375',
} # L1000 cell line : single-cell cell line

# - read adata_L1000, this is processed data
adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')
adata_L1000

AnnData object with n_obs × n_vars = 36720 × 978
    obs: 'sig_id', 'pert_id', 'pert_iname', 'pert_type', 'cell_id', 'pert_dose', 'pert_dose_unit', 'pert_idose', 'pert_time', 'pert_time_unit', 'pert_itime', 'distil_id'
    var: 'pr_gene_id', 'pr_gene_symbol', 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'

In [9]:
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, auc

for cell_line_bulk in list(common_cell_line.keys())[:]:
    cell_line_single = common_cell_line[cell_line_bulk]
    print('='*20, f'cell line is {cell_line_single}')
    
    #####################################################
    
    #===================prepare data
    if cell_line_bulk in ['PC3', 'A375']:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/SCP542'
    else:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/CNP0003658'
    adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))

    # - consctrut corr mtx
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    # corr_mtx = np.corrcoef(adata_rna.X.T)
    
    # - get var_names
    var_names = list(adata_rna.var_names)
    
    # - get common pert
    adata_L1000_sub = adata_L1000[adata_L1000.obs['cell_id']==cell_line_bulk]
    L1000_total_perts = np.unique(adata_L1000_sub.obs['pert_iname'])
    
    
    n_cells_downsample = 10000
    threshold_number = 10000
    
    ##########################################################
    
    # - get control adata
    adata = adata_rna.copy()
    adata.obs['celltype'] = cell_line_bulk
    print(f'adata.shape is: ',adata.shape)

    # -- get the baseGRN
    # Load TF info which was made from mouse cell atlas dataset.
    base_GRN = co.data.load_human_promoter_base_GRN()
    print('base_GRN.shape: ', base_GRN.shape)

    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/L1000'
    save_prefix = f'CellOracle/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(tmp_dir, save_prefix), exist_ok=True)

    save_dir = os.path.join(tmp_dir, save_prefix)
    if os.path.exists(os.path.join(save_dir, "ctrl.celloracle.oracle")):
        print('file exists')
        oracle = co.load_hdf5(os.path.join(save_dir, "ctrl.celloracle.oracle"))
        links = co.load_hdf5(file_path=os.path.join(save_dir, "ctrl.celloracle.links"))
        
    else:

        # - start CellOracle process for the whole ctrl

        # -- keep raw cont data before log transformation
        adata.raw = adata
        if not isinstance(adata.raw.X, np.ndarray):
            adata.layers["raw_count"] = (np.exp(adata.raw.X.toarray())-1).copy()
        else:
            adata.layers["raw_count"] = (np.exp(adata.raw.X)-1).copy()
            
        # -- get umap 
        sc.pp.scale(adata)
        # PCA
        sc.tl.pca(adata, svd_solver='arpack', random_state=2022)
        # UMAP
        sc.pp.neighbors(adata, n_neighbors=4, n_pcs=20, random_state=2022)
        sc.tl.umap(adata,random_state=2022)

        # -- Random downsampling into 30K cells if the anndata object include more than 30 K cells.
        if adata.shape[0] > n_cells_downsample:
            # Let's dowmsample into 30K cells
            sc.pp.subsample(adata, n_obs=n_cells_downsample, random_state=123)
        print(f"Cell number is :{adata.shape[0]}")

        # -- Instantiate Oracle object
        oracle = co.Oracle()

        # -- Check data in anndata
        print("Metadata columns :", list(adata.obs.columns))
        print("Dimensional reduction: ", list(adata.obsm.keys()))

        # -- In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
        adata.X = adata.layers["raw_count"].copy()

        # -- Instantiate Oracle object.
        oracle.import_anndata_as_raw_count(adata=adata,
                                        cluster_column_name="celltype",
                                        embedding_name="X_umap")

        # -- You can load TF info dataframe with the following code.
        oracle.import_TF_data(TF_info_matrix=base_GRN)

        # -- knn imputation, this step is needed for the whole ctrl
        # Perform PCA
        oracle.perform_PCA()

        # Select important PCs
        plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_)[:100])
        n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
        plt.axvline(n_comps, c="k")
        plt.show()
        print(n_comps)
        n_comps = min(n_comps, 50)

        n_cell = oracle.adata.shape[0]
        print(f"cell number is :{n_cell}")

        k = int(0.025*n_cell)
        print(f"Auto-selected k is :{k}")

        oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8,
                            b_maxl=k*4, n_jobs=4)

        # model_prefix = ''
        os.makedirs(save_dir, exist_ok=True)
        # -- save the oracle
        oracle.to_hdf5(os.path.join(save_dir, "ctrl.celloracle.oracle"))

        # -- get the links
        # Calculate GRN for each population in "louvain_annot" clustering unit.
        # This step may take some time.(~30 minutes)
        links = oracle.get_links(cluster_name_for_GRN_unit="celltype", alpha=10,
                                verbose_level=10)

        # -- Save Links object.
        links.to_hdf5(file_path=os.path.join(save_dir, "ctrl.celloracle.links"))


    # -- filter and get the coef_mtx
    links.filter_links(threshold_number=threshold_number,
                        p=0.001,
                        weight='coef_abs')
    oracle.get_cluster_specific_TFdict_from_Links(links_object=links)
    oracle.fit_GRN_for_simulation(alpha=10,
                                use_cluster_specific_TFdict=True)
    
    ###################################################
    # - get all the TFs in the base_GRN
    TFdict = import_TF_data(TF_info_matrix=base_GRN)
    tf_target_dict = {}
    for target, gene_set in TFdict.items():
        for tf in gene_set:
            if tf not in tf_target_dict:
                tf_target_dict[tf] = []
                tf_target_dict[tf].append(target)
            else:
                tf_target_dict[tf].append(target)
    total_tf_list = list(tf_target_dict.keys())
    


    
    #####################################################
    var_names = list(adata.var_names)    
    single_total_perts = np.intersect1d(total_tf_list, adata.var_names)
    common_perts = np.intersect1d(single_total_perts, L1000_total_perts)
    print('L1000_total_perts num: ', len(L1000_total_perts))
    print('common_perts num: ', len(common_perts))
    print('common var to L1000 data is: ', len(np.intersect1d(var_names, adata_L1000.var_names)))

    ###########################################
    celltype = adata.obs['celltype'].unique()[0]

    # - get the tf_GRN_dict, to check whether pert have regulatory relations
    gene_GRN_mtx = oracle.coef_matrix_per_cluster[celltype].copy()
    tf_GRN_mtx = gene_GRN_mtx[~(gene_GRN_mtx == 0).all(axis=1)]
    # - get TF-target pair and the regulatory values
    tf_GRN_dict = {} # the tf to targets
    for i in range(len(tf_GRN_mtx)):
        tmp = tf_GRN_mtx.iloc[i,:]
        tmp = tmp[tmp!=0]

        tf_GRN_dict[tf_GRN_mtx.index[i]] = {}
        for j in range(len(tmp)):
            tf_GRN_dict[tf_GRN_mtx.index[i]][tmp.index[j]] = tmp.values[j]

    ###########################################        
    # - get oracle_ctrl
    adata_rna.obs['celltype'] = cell_line_bulk
    adata_ctrl = adata_rna.copy()
    # keep raw cont data before log transformation
    adata_ctrl.raw = adata_ctrl

    # the result will be recovered in normalized_count
    if not isinstance(adata_ctrl.raw.X, np.ndarray):
        adata_ctrl.layers["raw_count"] = (np.exp(adata_ctrl.raw.X.toarray())-1).copy()
    else:
        adata_ctrl.layers["raw_count"] = (np.exp(adata_ctrl.raw.X)-1).copy()
        
    sc.pp.scale(adata_ctrl)
    # PCA
    sc.tl.pca(adata_ctrl, svd_solver='arpack', random_state=2022)

    # Diffusion map
    sc.pp.neighbors(adata_ctrl, n_neighbors=4, n_pcs=20, random_state=2022)
    sc.tl.umap(adata_ctrl,random_state=2022)

    # Instantiate Oracle object
    oracle_ctrl = co.Oracle()

    # In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
    adata_ctrl.X = adata_ctrl.layers["raw_count"].copy()

    # Instantiate Oracle object.
    oracle_ctrl.import_anndata_as_raw_count(adata=adata_ctrl,
                                    cluster_column_name="celltype",
                                    embedding_name="X_umap")

    # You can load TF info dataframe with the following code.
    oracle_ctrl.import_TF_data(TF_info_matrix=base_GRN)

    # get the imputed_count, here we dont do the impute to get the prediction
    oracle_ctrl.adata.layers["imputed_count"] = oracle_ctrl.adata.layers["normalized_count"].copy()

    # get the coef from the whole ctrl
    oracle_ctrl.coef_matrix_per_cluster = oracle.coef_matrix_per_cluster
    
    pert_gene_rank_dict = {} 
    for pert in tqdm(common_perts):
        

        # - this is for crispra
        gois = [pert]
        goi_dict = {}

        # - all data in L1000 is knockdown
        for goi in gois:
            # -- if original value is zero
            if np.mean(adata_rna[:,goi].X.toarray())==0:
                print(f'{goi} ctrl expression is 0')
                continue
            # -- if the TF has no targets
            if goi not in list(tf_GRN_dict.keys()):
                print(f'{goi} is not in the tf_GRN_dict, no targets')
                continue
            goi_dict[goi] = 0
        if len(goi_dict) == 0:
            print(f'{pert} is filtered')
            continue


        # Enter perturbation conditions to simulate signal propagation after the perturbation.
        oracle_ctrl.simulate_shift(perturb_condition=goi_dict,
                            n_propagation=3)
        # - get the prediction; delta_X = simulated_count - imputed_count
        delta_X, simulated_count = oracle_ctrl.adata.layers["delta_X"], oracle_ctrl.adata.layers["simulated_count"]


        # - create adata_pert
        adata_pert = adata_rna.copy()
        adata_pert.X = simulated_count
        adata_pert.X[adata_pert.X < 0] = 0
        adata_pert.obs_names = [i+f'_{pert}' for i in adata_pert.obs_names]

        # - adata_ctrl
        adata_ctrl = adata_rna.copy()

        adata_ctrl.obs['batch'] = 'ctrl'
        adata_pert.obs['batch'] = 'pert'

        adata_concat = ad.concat([adata_ctrl, adata_pert])
        adata_concat.obs['batch'] = adata_concat.obs['batch'].astype('category') 
        adata_concat.obs['celltype'] = adata_concat.obs['celltype'].astype('category') 

        # - cal de genes
        rankby_abs = False

        sc.tl.rank_genes_groups(
            adata_concat,
            groupby='batch',
            reference='ctrl',
            rankby_abs=rankby_abs,
            n_genes=len(adata_concat.var),
            use_raw=False,
            method = 'wilcoxon'
        )
        de_genes = pd.DataFrame(adata_concat.uns['rank_genes_groups']['names'])
        pvals = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals'])
        pvals_adj = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals_adj'])
        scores = pd.DataFrame(adata_concat.uns['rank_genes_groups']['scores'])
        logfoldchanges = pd.DataFrame(adata_concat.uns['rank_genes_groups']['logfoldchanges'])

        # - get gene_score
        gene_score = pd.DataFrame({'gene':list(de_genes['pert']),
                                    'z-score':list(scores['pert'])})

        pert_gene_rank_dict[pert] = (list(de_genes['pert']), list(scores['pert']))
        
        break
        
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/zero_shot/result'
    save_prefix = f'CellOracle/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(save_dir, save_prefix), exist_ok=True)

    import json
    with open(os.path.join(save_dir, save_prefix, 'pert_gene_rank_dict.json'), 'w') as f:
        json.dump(pert_gene_rank_dict, f)
        

    break

==================== cell line is A549
adata.shape is:  (500, 5155)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
base_GRN.shape:  (37003, 1096)
file exists


  0%|          | 0/1 [00:00<?, ?it/s]

L1000_total_perts num:  3620
common_perts num:  306
common var to L1000 data is:  933
5155 genes were found in the adata. Note that Celloracle is intended to use around 1000-3000 genes, so the behavior with this number of genes may differ from what is expected.


... storing 'celltype' as categorical
  0%|          | 0/306 [00:00<?, ?it/s]... storing 'celltype' as categorical
... storing 'batch' as categorical
  0%|          | 0/306 [01:14<?, ?it/s]


# L1000运行CellOracle-并行版

In [14]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, auc
from scipy.spatial.distance import cdist
import concurrent.futures
import json

# 定义处理每个 cell_line_single 的函数
def process_cell_line(cell_line_bulk, cell_line_single, common_cell_line, adata_L1000):
    print('=' * 20, f'cell line is {cell_line_single}')

    #===================prepare data
    if cell_line_bulk in ['PC3', 'A375']:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/SCP542'
    else:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/CNP0003658'
    adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))

    # - consctrut corr mtx
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    # corr_mtx = np.corrcoef(adata_rna.X.T)
    
    # - get var_names
    var_names = list(adata_rna.var_names)
    
    # - get common pert
    adata_L1000_sub = adata_L1000[adata_L1000.obs['cell_id']==cell_line_bulk]
    L1000_total_perts = np.unique(adata_L1000_sub.obs['pert_iname'])
    
    
    n_cells_downsample = 10000
    threshold_number = 10000
    
    ##########################################################
    
    # - get control adata
    adata = adata_rna.copy()
    adata.obs['celltype'] = cell_line_bulk
    print(f'adata.shape is: ',adata.shape)

    # -- get the baseGRN
    # Load TF info which was made from mouse cell atlas dataset.
    base_GRN = co.data.load_human_promoter_base_GRN()
    print('base_GRN.shape: ', base_GRN.shape)

    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/L1000'
    save_prefix = f'CellOracle/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(tmp_dir, save_prefix), exist_ok=True)

    save_dir = os.path.join(tmp_dir, save_prefix)
    if os.path.exists(os.path.join(save_dir, "ctrl.celloracle.oracle")):
        print('file exists')
        oracle = co.load_hdf5(os.path.join(save_dir, "ctrl.celloracle.oracle"))
        links = co.load_hdf5(file_path=os.path.join(save_dir, "ctrl.celloracle.links"))
        
    else:

        # - start CellOracle process for the whole ctrl

        # -- keep raw cont data before log transformation
        adata.raw = adata
        if not isinstance(adata.raw.X, np.ndarray):
            adata.layers["raw_count"] = (np.exp(adata.raw.X.toarray())-1).copy()
        else:
            adata.layers["raw_count"] = (np.exp(adata.raw.X)-1).copy()
            
        # -- get umap 
        sc.pp.scale(adata)
        # PCA
        sc.tl.pca(adata, svd_solver='arpack', random_state=2022)
        # UMAP
        sc.pp.neighbors(adata, n_neighbors=4, n_pcs=20, random_state=2022)
        sc.tl.umap(adata,random_state=2022)

        # -- Random downsampling into 30K cells if the anndata object include more than 30 K cells.
        if adata.shape[0] > n_cells_downsample:
            # Let's dowmsample into 30K cells
            sc.pp.subsample(adata, n_obs=n_cells_downsample, random_state=123)
        print(f"Cell number is :{adata.shape[0]}")

        # -- Instantiate Oracle object
        oracle = co.Oracle()

        # -- Check data in anndata
        print("Metadata columns :", list(adata.obs.columns))
        print("Dimensional reduction: ", list(adata.obsm.keys()))

        # -- In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
        adata.X = adata.layers["raw_count"].copy()

        # -- Instantiate Oracle object.
        oracle.import_anndata_as_raw_count(adata=adata,
                                        cluster_column_name="celltype",
                                        embedding_name="X_umap")

        # -- You can load TF info dataframe with the following code.
        oracle.import_TF_data(TF_info_matrix=base_GRN)

        # -- knn imputation, this step is needed for the whole ctrl
        # Perform PCA
        oracle.perform_PCA()

        # Select important PCs
        plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_)[:100])
        n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
        plt.axvline(n_comps, c="k")
        plt.show()
        print(n_comps)
        n_comps = min(n_comps, 50)

        n_cell = oracle.adata.shape[0]
        print(f"cell number is :{n_cell}")

        k = int(0.025*n_cell)
        print(f"Auto-selected k is :{k}")

        oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8,
                            b_maxl=k*4, n_jobs=4)

        # model_prefix = ''
        os.makedirs(save_dir, exist_ok=True)
        # -- save the oracle
        oracle.to_hdf5(os.path.join(save_dir, "ctrl.celloracle.oracle"))

        # -- get the links
        # Calculate GRN for each population in "louvain_annot" clustering unit.
        # This step may take some time.(~30 minutes)
        links = oracle.get_links(cluster_name_for_GRN_unit="celltype", alpha=10,
                                verbose_level=10)

        # -- Save Links object.
        links.to_hdf5(file_path=os.path.join(save_dir, "ctrl.celloracle.links"))


    # -- filter and get the coef_mtx
    links.filter_links(threshold_number=threshold_number,
                        p=0.001,
                        weight='coef_abs')
    oracle.get_cluster_specific_TFdict_from_Links(links_object=links)
    oracle.fit_GRN_for_simulation(alpha=10,
                                use_cluster_specific_TFdict=True)
    
    ###################################################
    # - get all the TFs in the base_GRN
    TFdict = import_TF_data(TF_info_matrix=base_GRN)
    tf_target_dict = {}
    for target, gene_set in TFdict.items():
        for tf in gene_set:
            if tf not in tf_target_dict:
                tf_target_dict[tf] = []
                tf_target_dict[tf].append(target)
            else:
                tf_target_dict[tf].append(target)
    total_tf_list = list(tf_target_dict.keys())
    


    
    #####################################################
    var_names = list(adata.var_names)    
    single_total_perts = np.intersect1d(total_tf_list, adata.var_names)
    common_perts = np.intersect1d(single_total_perts, L1000_total_perts)
    print('L1000_total_perts num: ', len(L1000_total_perts))
    print('common_perts num: ', len(common_perts))
    print('common var to L1000 data is: ', len(np.intersect1d(var_names, adata_L1000.var_names)))

    ###########################################
    celltype = adata.obs['celltype'].unique()[0]

    # - get the tf_GRN_dict, to check whether pert have regulatory relations
    gene_GRN_mtx = oracle.coef_matrix_per_cluster[celltype].copy()
    tf_GRN_mtx = gene_GRN_mtx[~(gene_GRN_mtx == 0).all(axis=1)]
    # - get TF-target pair and the regulatory values
    tf_GRN_dict = {} # the tf to targets
    for i in range(len(tf_GRN_mtx)):
        tmp = tf_GRN_mtx.iloc[i,:]
        tmp = tmp[tmp!=0]

        tf_GRN_dict[tf_GRN_mtx.index[i]] = {}
        for j in range(len(tmp)):
            tf_GRN_dict[tf_GRN_mtx.index[i]][tmp.index[j]] = tmp.values[j]

    ###########################################        
    # - get oracle_ctrl
    adata_rna.obs['celltype'] = cell_line_bulk
    adata_ctrl = adata_rna.copy()
    # keep raw cont data before log transformation
    adata_ctrl.raw = adata_ctrl

    # the result will be recovered in normalized_count
    if not isinstance(adata_ctrl.raw.X, np.ndarray):
        adata_ctrl.layers["raw_count"] = (np.exp(adata_ctrl.raw.X.toarray())-1).copy()
    else:
        adata_ctrl.layers["raw_count"] = (np.exp(adata_ctrl.raw.X)-1).copy()
        
    sc.pp.scale(adata_ctrl)
    # PCA
    sc.tl.pca(adata_ctrl, svd_solver='arpack', random_state=2022)

    # Diffusion map
    sc.pp.neighbors(adata_ctrl, n_neighbors=4, n_pcs=20, random_state=2022)
    sc.tl.umap(adata_ctrl,random_state=2022)

    # Instantiate Oracle object
    oracle_ctrl = co.Oracle()

    # In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
    adata_ctrl.X = adata_ctrl.layers["raw_count"].copy()

    # Instantiate Oracle object.
    oracle_ctrl.import_anndata_as_raw_count(adata=adata_ctrl,
                                    cluster_column_name="celltype",
                                    embedding_name="X_umap")

    # You can load TF info dataframe with the following code.
    oracle_ctrl.import_TF_data(TF_info_matrix=base_GRN)

    # get the imputed_count, here we dont do the impute to get the prediction
    oracle_ctrl.adata.layers["imputed_count"] = oracle_ctrl.adata.layers["normalized_count"].copy()

    # get the coef from the whole ctrl
    oracle_ctrl.coef_matrix_per_cluster = oracle.coef_matrix_per_cluster
    
    pert_gene_rank_dict = {} 
    for pert in tqdm(common_perts):
        

        # - this is for crispra
        gois = [pert]
        goi_dict = {}

        # - all data in L1000 is knockdown
        for goi in gois:
            # -- if original value is zero
            if np.mean(adata_rna[:,goi].X.toarray())==0:
                print(f'{goi} ctrl expression is 0')
                continue
            # -- if the TF has no targets
            if goi not in list(tf_GRN_dict.keys()):
                print(f'{goi} is not in the tf_GRN_dict, no targets')
                continue
            goi_dict[goi] = 0
        if len(goi_dict) == 0:
            print(f'{pert} is filtered')
            continue


        # Enter perturbation conditions to simulate signal propagation after the perturbation.
        oracle_ctrl.simulate_shift(perturb_condition=goi_dict,
                            n_propagation=3)
        # - get the prediction; delta_X = simulated_count - imputed_count
        delta_X, simulated_count = oracle_ctrl.adata.layers["delta_X"], oracle_ctrl.adata.layers["simulated_count"]


        # - create adata_pert
        adata_pert = adata_rna.copy()
        adata_pert.X = simulated_count
        adata_pert.X[adata_pert.X < 0] = 0
        adata_pert.obs_names = [i+f'_{pert}' for i in adata_pert.obs_names]

        # - adata_ctrl
        adata_ctrl = adata_rna.copy()

        adata_ctrl.obs['batch'] = 'ctrl'
        adata_pert.obs['batch'] = 'pert'

        adata_concat = ad.concat([adata_ctrl, adata_pert])
        adata_concat.obs['batch'] = adata_concat.obs['batch'].astype('category') 
        adata_concat.obs['celltype'] = adata_concat.obs['celltype'].astype('category') 

        # - cal de genes
        rankby_abs = False

        sc.tl.rank_genes_groups(
            adata_concat,
            groupby='batch',
            reference='ctrl',
            rankby_abs=rankby_abs,
            n_genes=len(adata_concat.var),
            use_raw=False,
            method = 'wilcoxon'
        )
        de_genes = pd.DataFrame(adata_concat.uns['rank_genes_groups']['names'])
        pvals = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals'])
        pvals_adj = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals_adj'])
        scores = pd.DataFrame(adata_concat.uns['rank_genes_groups']['scores'])
        logfoldchanges = pd.DataFrame(adata_concat.uns['rank_genes_groups']['logfoldchanges'])

        # - get gene_score
        gene_score = pd.DataFrame({'gene':list(de_genes['pert']),
                                    'z-score':list(scores['pert'])})

        pert_gene_rank_dict[pert] = (list(de_genes['pert']), list(scores['pert']))
        
        # break
        
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/zero_shot/result'
    save_prefix = f'CellOracle/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(save_dir, save_prefix), exist_ok=True)

    import json
    with open(os.path.join(save_dir, save_prefix, 'pert_gene_rank_dict.json'), 'w') as f:
        json.dump(pert_gene_rank_dict, f)
        
# 主函数
if __name__ == "__main__":
    # - get cell line name
    common_cell_line = \
    {   'A549': 'A549',
        'HEPG2': 'HepG2',
        'HT29': 'HT29',
        'MCF7': 'MCF7',
        # 'SKBR3': 'SK-BR-3',
        'SW480': 'SW480',
        'PC3': 'PC3',
        'A375': 'A375',
    } # L1000 cell line : single-cell cell line

    # - read adata_L1000, this is processed data
    adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')


    # 使用并行执行
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(process_cell_line, cell_line_bulk, common_cell_line[cell_line_bulk], common_cell_line, adata_L1000)
            for cell_line_bulk in common_cell_line.keys()
        ]
        
        # 等待所有任务完成
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # 获取每个任务的结果，如果有异常，将在此处抛出
            except Exception as e:
                print(f"Error: {e}")


==================== cell line is A549
adata.shape is:  (500, 5155)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
==================== cell line is HepG2
base_GRN.shape:  (37003, 1096)
file exists
====================adata.shape is:   cell line is HT29(500, 5032)

Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
base_GRN.shape:  (37003, 1096)
file exists
adata.shape is:  (500, 5141)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
==================== cell line is MCF7
base_GRN.shape:  (37003, 1096)
file exists
adata.shape is:  (409, 5450)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2

==================== cell line is SW480
base_GRN.shape:  (37003, 1096)
file exists==================== cell line is PC3
adata.shape is:  (169, 5417)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
adata.shape is:  (500, 3687)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
=====

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

... storing 'celltype' as categorical
  1%|          | 2/278 [01:07<2:32:51, 33.23s/it]

ARHGEF12 is not in the tf_GRN_dict, no targets
ARHGEF12 is filtered


  1%|▏         | 4/278 [01:38<1:40:45, 22.06s/it]

ARID5B is not in the tf_GRN_dict, no targets
ARID5B is filtered


  0%|          | 1/280 [01:40<7:49:00, 100.86s/it]

# L1000运行CellOracle-并行版 - v2 使用delta直接加上去

In [7]:
save_prefix_method = 'CellOracle_v2'

# CellOracle: use simulated_count
# CellOracle_v2: use delte_count

In [8]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, auc
from scipy.spatial.distance import cdist
import concurrent.futures
import json

# 定义处理每个 cell_line_single 的函数
def process_cell_line(cell_line_bulk, cell_line_single, common_cell_line, adata_L1000):
    print('=' * 20, f'cell line is {cell_line_single}')

    #===================prepare data
    if cell_line_bulk in ['PC3', 'A375']:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/SCP542'
    else:
        save_dir_adata = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/single_cell_data/CNP0003658'
    adata_rna = sc.read(os.path.join(save_dir_adata, cell_line_bulk, f'adata_{cell_line_bulk}.h5ad'))

    # - consctrut corr mtx
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    # corr_mtx = np.corrcoef(adata_rna.X.T)
    
    # - get var_names
    var_names = list(adata_rna.var_names)
    
    # - get common pert
    adata_L1000_sub = adata_L1000[adata_L1000.obs['cell_id']==cell_line_bulk]
    L1000_total_perts = np.unique(adata_L1000_sub.obs['pert_iname'])
    
    
    n_cells_downsample = 10000
    threshold_number = 10000
    
    ##########################################################
    
    # - get control adata
    adata = adata_rna.copy()
    adata.obs['celltype'] = cell_line_bulk
    print(f'adata.shape is: ',adata.shape)

    # -- get the baseGRN
    # Load TF info which was made from mouse cell atlas dataset.
    base_GRN = co.data.load_human_promoter_base_GRN()
    print('base_GRN.shape: ', base_GRN.shape)

    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/L1000'
    save_prefix = f'CellOracle/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(tmp_dir, save_prefix), exist_ok=True)

    save_dir = os.path.join(tmp_dir, save_prefix)
    if os.path.exists(os.path.join(save_dir, "ctrl.celloracle.oracle")):
        print('file exists')
        oracle = co.load_hdf5(os.path.join(save_dir, "ctrl.celloracle.oracle"))
        links = co.load_hdf5(file_path=os.path.join(save_dir, "ctrl.celloracle.links"))
        
    else:

        # - start CellOracle process for the whole ctrl

        # -- keep raw cont data before log transformation
        adata.raw = adata
        if not isinstance(adata.raw.X, np.ndarray):
            adata.layers["raw_count"] = (np.exp(adata.raw.X.toarray())-1).copy()
        else:
            adata.layers["raw_count"] = (np.exp(adata.raw.X)-1).copy()
            
        # -- get umap 
        sc.pp.scale(adata)
        # PCA
        sc.tl.pca(adata, svd_solver='arpack', random_state=2022)
        # UMAP
        sc.pp.neighbors(adata, n_neighbors=4, n_pcs=20, random_state=2022)
        sc.tl.umap(adata,random_state=2022)

        # -- Random downsampling into 30K cells if the anndata object include more than 30 K cells.
        if adata.shape[0] > n_cells_downsample:
            # Let's dowmsample into 30K cells
            sc.pp.subsample(adata, n_obs=n_cells_downsample, random_state=123)
        print(f"Cell number is :{adata.shape[0]}")

        # -- Instantiate Oracle object
        oracle = co.Oracle()

        # -- Check data in anndata
        print("Metadata columns :", list(adata.obs.columns))
        print("Dimensional reduction: ", list(adata.obsm.keys()))

        # -- In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
        adata.X = adata.layers["raw_count"].copy()

        # -- Instantiate Oracle object.
        oracle.import_anndata_as_raw_count(adata=adata,
                                        cluster_column_name="celltype",
                                        embedding_name="X_umap")

        # -- You can load TF info dataframe with the following code.
        oracle.import_TF_data(TF_info_matrix=base_GRN)

        # -- knn imputation, this step is needed for the whole ctrl
        # Perform PCA
        oracle.perform_PCA()

        # Select important PCs
        plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_)[:100])
        n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
        plt.axvline(n_comps, c="k")
        plt.show()
        print(n_comps)
        n_comps = min(n_comps, 50)

        n_cell = oracle.adata.shape[0]
        print(f"cell number is :{n_cell}")

        k = int(0.025*n_cell)
        print(f"Auto-selected k is :{k}")

        oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8,
                            b_maxl=k*4, n_jobs=4)

        # model_prefix = ''
        os.makedirs(save_dir, exist_ok=True)
        # -- save the oracle
        oracle.to_hdf5(os.path.join(save_dir, "ctrl.celloracle.oracle"))

        # -- get the links
        # Calculate GRN for each population in "louvain_annot" clustering unit.
        # This step may take some time.(~30 minutes)
        links = oracle.get_links(cluster_name_for_GRN_unit="celltype", alpha=10,
                                verbose_level=10)

        # -- Save Links object.
        links.to_hdf5(file_path=os.path.join(save_dir, "ctrl.celloracle.links"))


    # -- filter and get the coef_mtx
    links.filter_links(threshold_number=threshold_number,
                        p=0.001,
                        weight='coef_abs')
    oracle.get_cluster_specific_TFdict_from_Links(links_object=links)
    oracle.fit_GRN_for_simulation(alpha=10,
                                use_cluster_specific_TFdict=True)
    
    ###################################################
    # - get all the TFs in the base_GRN
    TFdict = import_TF_data(TF_info_matrix=base_GRN)
    tf_target_dict = {}
    for target, gene_set in TFdict.items():
        for tf in gene_set:
            if tf not in tf_target_dict:
                tf_target_dict[tf] = []
                tf_target_dict[tf].append(target)
            else:
                tf_target_dict[tf].append(target)
    total_tf_list = list(tf_target_dict.keys())
    


    
    #####################################################
    var_names = list(adata.var_names)    
    single_total_perts = np.intersect1d(total_tf_list, adata.var_names)
    common_perts = np.intersect1d(single_total_perts, L1000_total_perts)
    print('L1000_total_perts num: ', len(L1000_total_perts))
    print('common_perts num: ', len(common_perts))
    print('common var to L1000 data is: ', len(np.intersect1d(var_names, adata_L1000.var_names)))

    ###########################################
    celltype = adata.obs['celltype'].unique()[0]

    # - get the tf_GRN_dict, to check whether pert have regulatory relations
    gene_GRN_mtx = oracle.coef_matrix_per_cluster[celltype].copy()
    tf_GRN_mtx = gene_GRN_mtx[~(gene_GRN_mtx == 0).all(axis=1)]
    # - get TF-target pair and the regulatory values
    tf_GRN_dict = {} # the tf to targets
    for i in range(len(tf_GRN_mtx)):
        tmp = tf_GRN_mtx.iloc[i,:]
        tmp = tmp[tmp!=0]

        tf_GRN_dict[tf_GRN_mtx.index[i]] = {}
        for j in range(len(tmp)):
            tf_GRN_dict[tf_GRN_mtx.index[i]][tmp.index[j]] = tmp.values[j]

    ###########################################        
    # - get oracle_ctrl
    adata_rna.obs['celltype'] = cell_line_bulk
    adata_ctrl = adata_rna.copy()
    # keep raw cont data before log transformation
    adata_ctrl.raw = adata_ctrl

    # the result will be recovered in normalized_count
    if not isinstance(adata_ctrl.raw.X, np.ndarray):
        adata_ctrl.layers["raw_count"] = (np.exp(adata_ctrl.raw.X.toarray())-1).copy()
    else:
        adata_ctrl.layers["raw_count"] = (np.exp(adata_ctrl.raw.X)-1).copy()
        
    sc.pp.scale(adata_ctrl)
    # PCA
    sc.tl.pca(adata_ctrl, svd_solver='arpack', random_state=2022)

    # Diffusion map
    sc.pp.neighbors(adata_ctrl, n_neighbors=4, n_pcs=20, random_state=2022)
    sc.tl.umap(adata_ctrl,random_state=2022)

    # Instantiate Oracle object
    oracle_ctrl = co.Oracle()

    # In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
    adata_ctrl.X = adata_ctrl.layers["raw_count"].copy()

    # Instantiate Oracle object.
    oracle_ctrl.import_anndata_as_raw_count(adata=adata_ctrl,
                                    cluster_column_name="celltype",
                                    embedding_name="X_umap")

    # You can load TF info dataframe with the following code.
    oracle_ctrl.import_TF_data(TF_info_matrix=base_GRN)

    # get the imputed_count, here we dont do the impute to get the prediction
    oracle_ctrl.adata.layers["imputed_count"] = oracle_ctrl.adata.layers["normalized_count"].copy()

    # get the coef from the whole ctrl
    oracle_ctrl.coef_matrix_per_cluster = oracle.coef_matrix_per_cluster
    
    pert_gene_rank_dict = {} 
    for pert in tqdm(common_perts):
        

        # - this is for crispra
        gois = [pert]
        goi_dict = {}

        # - all data in L1000 is knockdown
        for goi in gois:
            # -- if original value is zero
            if np.mean(adata_rna[:,goi].X.toarray())==0:
                print(f'{goi} ctrl expression is 0')
                continue
            # -- if the TF has no targets
            if goi not in list(tf_GRN_dict.keys()):
                print(f'{goi} is not in the tf_GRN_dict, no targets')
                continue
            goi_dict[goi] = 0
        if len(goi_dict) == 0:
            print(f'{pert} is filtered')
            continue


        # Enter perturbation conditions to simulate signal propagation after the perturbation.
        oracle_ctrl.simulate_shift(perturb_condition=goi_dict,
                            n_propagation=3)
        # - get the prediction; delta_X = simulated_count - imputed_count
        delta_X, simulated_count = oracle_ctrl.adata.layers["delta_X"], oracle_ctrl.adata.layers["simulated_count"]


        # - create adata_pert
        adata_pert = adata_rna.copy()
        if save_prefix_method == 'CellOracle':
            adata_pert.X = simulated_count
        if save_prefix_method == 'CellOracle_v2':
            adata_pert.X += delta_X
        adata_pert.X[adata_pert.X < 0] = 0
        adata_pert.obs_names = [i+f'_{pert}' for i in adata_pert.obs_names]

        # - adata_ctrl
        adata_ctrl = adata_rna.copy()

        adata_ctrl.obs['batch'] = 'ctrl'
        adata_pert.obs['batch'] = 'pert'

        adata_concat = ad.concat([adata_ctrl, adata_pert])
        adata_concat.obs['batch'] = adata_concat.obs['batch'].astype('category') 
        adata_concat.obs['celltype'] = adata_concat.obs['celltype'].astype('category') 

        # - cal de genes
        rankby_abs = False

        sc.tl.rank_genes_groups(
            adata_concat,
            groupby='batch',
            reference='ctrl',
            rankby_abs=rankby_abs,
            n_genes=len(adata_concat.var),
            use_raw=False,
            method = 'wilcoxon'
        )
        de_genes = pd.DataFrame(adata_concat.uns['rank_genes_groups']['names'])
        pvals = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals'])
        pvals_adj = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals_adj'])
        scores = pd.DataFrame(adata_concat.uns['rank_genes_groups']['scores'])
        logfoldchanges = pd.DataFrame(adata_concat.uns['rank_genes_groups']['logfoldchanges'])

        # - get gene_score
        gene_score = pd.DataFrame({'gene':list(de_genes['pert']),
                                    'z-score':list(scores['pert'])})

        pert_gene_rank_dict[pert] = (list(de_genes['pert']), list(scores['pert']))
        
        # break
        
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/zero_shot/result'
    save_prefix = f'{save_prefix_method}/{cell_line_bulk}' # use result of K562 to do the direct transfer
    os.makedirs(os.path.join(save_dir, save_prefix), exist_ok=True)

    import json
    with open(os.path.join(save_dir, save_prefix, 'pert_gene_rank_dict.json'), 'w') as f:
        json.dump(pert_gene_rank_dict, f)
        
# 主函数
if __name__ == "__main__":
    # - get cell line name
    common_cell_line = \
    {   'A549': 'A549',
        'HEPG2': 'HepG2',
        'HT29': 'HT29',
        'MCF7': 'MCF7',
        # 'SKBR3': 'SK-BR-3',
        'SW480': 'SW480',
        'PC3': 'PC3',
        'A375': 'A375',
    } # L1000 cell line : single-cell cell line

    # - read adata_L1000, this is processed data
    adata_L1000 = sc.read('/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/benchmark_data/L1000/GSE92742/adata_gene_pert.h5ad')


    # 使用并行执行
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(process_cell_line, cell_line_bulk, common_cell_line[cell_line_bulk], common_cell_line, adata_L1000)
            for cell_line_bulk in common_cell_line.keys()
        ]
        
        # 等待所有任务完成
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # 获取每个任务的结果，如果有异常，将在此处抛出
            except Exception as e:
                print(f"Error: {e}")


==================== cell line is A549
==================== cell line is HepG2
adata.shape is:  (500, 5155)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
==================== cell line is HT29
base_GRN.shape:  (37003, 1096)
file exists
==================== cell line is MCF7
adata.shape is:  (500, 5032)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
base_GRN.shape:  (37003, 1096)
file exists
adata.shape is:  (500, 5141)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
==================== cell line is SW480
base_GRN.shape:  (37003, 1096)
file exists
adata.shape is:  (409, 5450)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
base_GRN.shape:  (37003, 1096)
file exists
==================== cell line is PC3
adata.shape is:  (169, 5417)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
==================== cell line is A375
base_GRN.shape:  (37003, 1096)
file exists
adata.shape is:  (500

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

... storing 'x' as categorical
 97%|█████████▋| 281/291 [4:32:07<07:22, 44.30s/it]

ZNF490 is not in the tf_GRN_dict, no targets
ZNF490 is filtered


... storing 'x' as categorical
 98%|█████████▊| 286/291 [4:35:58<04:05, 49.08s/it]

ZNF582 is not in the tf_GRN_dict, no targets
ZNF582 is filtered
ZNF586 is not in the tf_GRN_dict, no targets
ZNF586 is filtered


... storing 'x' as categorical
100%|██████████| 291/291 [4:38:51<00:00, 57.50s/it]


# debug

In [12]:
for pert in tqdm(common_perts):
    

    # - this is for crispra
    gois = [pert]
    goi_dict = {}

    # - all data in L1000 is knockdown
    for goi in gois:
        # -- if original value is zero
        if np.mean(adata_rna[:,goi].X.toarray())==0:
            print(f'{goi} ctrl expression is 0')
            continue
        # -- if the TF has no targets
        if goi not in list(tf_GRN_dict.keys()):
            print(f'{goi} is not in the tf_GRN_dict, no targets')
            continue
        goi_dict[goi] = 0
    if len(goi_dict) == 0:
        print(f'{pert} is filtered')
        continue


    # Enter perturbation conditions to simulate signal propagation after the perturbation.
    oracle_ctrl.simulate_shift(perturb_condition=goi_dict,
                        n_propagation=3)
    # - get the prediction; delta_X = simulated_count - imputed_count
    delta_X, simulated_count = oracle_ctrl.adata.layers["delta_X"], oracle_ctrl.adata.layers["simulated_count"]


    # - create adata_pert
    adata_pert = adata_rna.copy()
    adata_pert.X = simulated_count
    adata_pert.X[adata_pert.X < 0] = 0
    adata_pert.obs_names = [i+f'_{pert}' for i in adata_pert.obs_names]

    # - adata_ctrl
    adata_ctrl = adata_rna.copy()

    adata_ctrl.obs['batch'] = 'ctrl'
    adata_pert.obs['batch'] = 'pert'

    adata_concat = ad.concat([adata_ctrl, adata_pert])
    adata_concat.obs['batch'] = adata_concat.obs['batch'].astype('category') 
    adata_concat.obs['celltype'] = adata_concat.obs['celltype'].astype('category') 

    # - cal de genes
    rankby_abs = False

    sc.tl.rank_genes_groups(
        adata_concat,
        groupby='batch',
        reference='ctrl',
        rankby_abs=rankby_abs,
        n_genes=len(adata_concat.var),
        use_raw=False,
        method = 'wilcoxon'
    )
    de_genes = pd.DataFrame(adata_concat.uns['rank_genes_groups']['names'])
    pvals = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals'])
    pvals_adj = pd.DataFrame(adata_concat.uns['rank_genes_groups']['pvals_adj'])
    scores = pd.DataFrame(adata_concat.uns['rank_genes_groups']['scores'])
    logfoldchanges = pd.DataFrame(adata_concat.uns['rank_genes_groups']['logfoldchanges'])

    # - get gene_score
    gene_score = pd.DataFrame({'gene':list(de_genes['pert']),
                                'z-score':list(scores['pert'])})

    pert_gene_rank_dict[pert] = (list(de_genes['pert']), list(scores['pert']))
    
    # break

  0%|          | 0/306 [00:00<?, ?it/s]

  1%|          | 2/306 [02:51<7:14:43, 85.80s/it]


KeyboardInterrupt: 

In [ ]:
sc.tl.rank_genes_groups(
    adata_concat,
    groupby='batch',
    reference='ctrl',
    rankby_abs=rankby_abs,
    n_genes=len(adata_concat.var),
    use_raw=False,
    method = 'wilcoxon'
)